# VacationPy
----
#### Author: Felipe Murillo
#### Date: May 2, 2020
#### Description: Use weather data from 500+ cities across the world to plan the perfect vacation
#### Inputs:  ./WeatherPy/output_data/cities.csv
----

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import sys

# Import API key
sys.path.append('../')
from WeatherPy.config import gkey


### Input City Weather DataFrame


In [2]:
#Read in city weather data
city_weather_df = pd.read_csv("../WeatherPy/output_data/cities.csv",index_col=0)
city_weather_df.head()

,City,City Name,Country,Lng,Lat,Max Temp (F),Humidity (%),Cloudiness (%),Wind Speed (mph)
0,vyazemskiy,Vyazemskiy,RU,134.75,47.54,47.19,59,58,0.65
1,sisimiut,Sisimiut,GL,-53.67,66.94,25.29,92,100,8.84
2,barrow,Barrow,US,-156.79,71.29,8.60,84,1,11.41
3,zhigansk,Zhigansk,RU,123.37,66.77,28.80,95,80,9.89
4,vaini,Vaini,TO,-175.20,-21.20,80.60,78,40,5.82


In [3]:
# Store latitude and longitude in locations
city_locations = city_weather_df[["Lat","Lng"]]
city_locations.head()

,Lat,Lng
0,47.54,134.75
1,66.94,-53.67
2,71.29,-156.79
3,66.77,123.37
4,-21.20,-175.20


### Humidity Heatmap

In [4]:
#Configure gmaps
gmaps.configure(api_key=gkey)

In [5]:
# Plot humidity heatmap
fig = gmaps.figure(map_type="HYBRID",
                   center=(0, 0), 
                   zoom_level=1
                  )
heatmap_layer = gmaps.heatmap_layer(
    city_locations,
    weights = city_weather_df["Humidity (%)"],
    max_intensity = 150,
    point_radius = 10
)
fig.add_layer(heatmap_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria

In [6]:
# Create a filter for the following conditions
# A max temperature lower than 80 degrees but higher than 70.
# Wind speed less than 10 mph.
# Zero cloudiness
max_ideal_temp = 80 # deg F
min_ideal_temp = 70 # deg F
max_ideal_wind = 10 # mph
max_ideal_cloud = 0 # cloudiness (%)

perfect_weather = (city_weather_df["Max Temp (F)"] > min_ideal_temp) & \
                  (city_weather_df["Max Temp (F)"] < max_ideal_temp) & \
                  (city_weather_df["Wind Speed (mph)"] < max_ideal_wind) & \
                  (city_weather_df["Cloudiness (%)"] <= max_ideal_cloud)
ideal_cities = city_weather_df.loc[perfect_weather].reset_index()
ideal_cities

,index,City,City Name,Country,Lng,Lat,Max Temp (F),Humidity (%),Cloudiness (%),Wind Speed (mph)
0,182,namibe,Mossamedes,AO,12.15,-15.20,77.52,59,0,1.88
1,260,tura,Tura,IN,90.22,25.52,72.28,68,0,3.78
2,429,taitung,Taitung City,TW,121.14,22.76,77.00,73,0,4.70
3,514,mlonggo,Mlonggo,ID,110.70,-6.53,78.69,84,0,4.38
4,552,benguela,Benguela,AO,13.41,-12.58,77.07,80,0,2.55


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
hotel_df = ideal_cities[["City Name","Country","Lat","Lng"]]
hotel_df["Hotel Name"] = ""
hotel_df["Hotel Lat"]= ""
hotel_df["Hotel Lng"]= ""
hotel_df

/Users/felipe/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/felipe/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/felipe/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try us

,City Name,Country,Lat,Lng,Hotel Name,Hotel Lat,Hotel Lng
0,Mossamedes,AO,-15.20,12.15,,,
1,Tura,IN,25.52,90.22,,,
2,Taitung City,TW,22.76,121.14,,,
3,Mlonggo,ID,-6.53,110.70,,,
4,Benguela,AO,-12.58,13.41,,,


In [8]:
# Base URL
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# use iterrows to iterate through pandas dataframe
for index, row in hotel_df.iterrows():

    # assemble url and make API request
    print(f"Retrieving Results for Index: {index}, hotel in {row['City Name']}")
    
    # geocoordinates
    target_coordinates = f"{row['Lat']}, {row['Lng']}"
    target_search = "Hotel"
    target_radius = 50000
    target_type = "hotel"
    
    # set up a parameters dictionary
    params = {
        'location': target_coordinates,
        'keyword': target_search,
        'radius': target_radius,
        'type': target_type,
        'key': gkey
        }
    
    # Pull data from API
    hotel_data = requests.get(base_url, params=params).json()
    
    # extract results
    results = hotel_data['results']
    
    try:
        hotel_df.loc[index, 'Hotel Name'] = results[0]["name"]
        hotel_df.loc[index, 'Hotel Lat'] = results[0]["geometry"]["location"]["lat"]
        hotel_df.loc[index, 'Hotel Lng'] = results[0]["geometry"]["location"]["lng"]
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")

Retrieving Results for Index: 0, hotel in Mossamedes


/Users/felipe/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


------------
Retrieving Results for Index: 1, hotel in Tura
------------
Retrieving Results for Index: 2, hotel in Taitung City
------------
Retrieving Results for Index: 3, hotel in Mlonggo
------------
Retrieving Results for Index: 4, hotel in Benguela
------------


In [9]:
# Show list of hotels in ideal cities and do a little clean up
hotel_df = hotel_df.rename(columns ={"Lat":"City Lat","Lng":"City Lng"})
hotel_df

,City Name,Country,City Lat,City Lng,Hotel Name,Hotel Lat,Hotel Lng
0,Mossamedes,AO,-15.20,12.15,iu Hotel Namibe,-15.1909,12.1513
1,Tura,IN,25.52,90.22,Hotel Polo Orchid,25.5145,90.1922
2,Taitung City,TW,22.76,121.14,巴斯拉岸民宿basrashore house,23.0997,121.373
3,Mlonggo,ID,-6.53,110.70,Quest Hotel,-6.97832,110.423
4,Benguela,AO,-12.58,13.41,iu Hotel Benguela,-12.6034,13.4032


In [10]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City Name}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# Create hotel locations
hotel_locations = [hotel_df.loc[hotel][['Hotel Lat','Hotel Lng']] for hotel in hotel_df.index]

# Create marker layer
marker_layer = gmaps.marker_layer(hotel_locations,info_box_content = hotel_info)

# Add marker layer ontop of heat map
fig.add_layer(marker_layer)

#Display Map
fig

Figure(layout=FigureLayout(height='420px'))